In [ ]:
import keras
import keras.backend as K
from keras.layers import RNN

from keras.preprocessing.sequence import TimeseriesGenerator

class MinimalRNNCell(keras.layers.Layer):
    def __init__(self,units):
        self.units = units
        self.state_size = units #??
        super(MinimalRNNCell,self).__init__()
        return
    
    def build(self,input_shape):
        self.kernel = self.add_weight(shape=(input_shape[-1],self.units),
                                      initializer='uniform', name='kernel')
        self.r_kernel = self.add_weight(shape=(self.units,self.units),
                                      initializer='uniform', name='r_kernel')
        self.built = True #??
        return
    
    def call(self,inputs,states):
        prev_out = states[0]
        h = K.dot(inputs,self.kernel)
        output = h+K.dot(prev_out,self.r_kernel)
        return output, [output]


In [ ]:
cell = [MinimalRNNCell(32), MinimalRNNCell(28)]
# or can be cells = []
# None - for time distribution?
x = keras.Input((None,5))
layer = RNN(cell,return_sequences=True)
y = layer(x)
from keras.models import Model
model = Model(inputs=[x],outputs=[y])
print (model.layers)

In [ ]:
##
## TimeseriesGenerator
##
import numpy as np
data = np.array([i for i in range(50)])
targets = np.array([-i-1 for i in range(50)])
data_gen = TimeseriesGenerator(data, targets,
                               length=10, sampling_rate=2,stride=2,
                               batch_size=2)
print (data_gen[0][0])
# call model.fit_generator

In [ ]:
## Task 1
## Research simple model, get gradients, different parameters
##

In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD,Adam

model_data = np.random.rand(100,5)
model_labels = np.random.rand(100,10)

model = Sequential((Dense(10,input_shape=(5,)),))
optimizer = SGD(lr=0.01)

model.compile(optimizer=optimizer,loss="mse")

model.fit(model_data,model_labels,batch_size=10,verbose=1,epochs=1,validation_split=0.1)

print (model.output)
print (model.trainable_weights)

In [ ]:
import keras.backend as K

gradients = K.gradients(model.total_loss,model.trainable_weights)
print (gradients)
print (model.total_loss)

input_tensors = [model.inputs[0], # input data
                 model.sample_weights[0], # how much to weight each sample by
                 model.targets[0], # labels
                 K.learning_phase(), # train or test mode
]
get_gradients = K.function(inputs=input_tensors, outputs=gradients)
inputs = [[model_data[0]],[1],[model_labels[0]],0]
print (get_gradients(inputs))

In [ ]:
import keras.backend as K

gradients = model.optimizer.get_gradients(model.total_loss,model.trainable_weights)
print (gradients)
print (model.total_loss)

input_tensors = [model.inputs[0], # input data
                 model.sample_weights[0], # how much to weight each sample by
                 model.targets[0], # labels
                 K.learning_phase(), # train or test mode
]
get_gradients = K.function(inputs=input_tensors, outputs=gradients)
inputs = [[model_data[0]],[1],[model_labels[0]],0]
print (get_gradients(inputs))

In [46]:
def getModelGradients(model,data,labels):
    """
    data - data, feeded to the model - [tens(input_shape),tens(input_shape)...]
    """
    grad_tens = K.gradients(model.total_loss,model.trainable_weights)
    #gradients = model.optimizer.get_gradients()
    input_tens = [model.inputs[0],model.sample_weights[0],model.targets[0],K.learning_phase()]
    get_gradients = K.function(inputs=input_tens,outputs=grad_tens)
    return get_gradients([data,len(data)*[1],labels,0])

In [ ]:
import keras.backend as K

gradients = model.optimizer.get_gradients(model.total_loss,model.trainable_weights)
print (gradients)
print (model.total_loss)

input_tensors = [model.inputs[0], # input data
                 model.sample_weights[0], # how much to weight each sample by
                 model.targets[0], # labels
                 K.learning_phase(), # train or test mode
]
get_gradients = K.function(inputs=input_tensors, outputs=gradients)
inputs = [[model_data[0]],[1],[model_labels[0]],0]
print (get_gradients(inputs))

In [ ]:
import keras.backend as K
import numpy as np

var = K.variable(value=np.array([1,2,3],dtype=np.float16))
res = var*var
print (res)

In [ ]:
fun = K.function(inputs=[],outputs=[res])
print (fun([]))

In [ ]:
K.eval(res)

In [ ]:
K.eval(K.constant(np.array([1,2,3,4])))

In [ ]:
a = K.constant(np.array([2,3]))
b = K.constant(np.array([[10,10],[20,20]]))
res = a*b
print (K.eval(res))
print (K.eval(K.sum(res,axis=1)))

In [49]:
import keras 
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Lambda,Dense
import numpy as np
from keras.initializers import Initializer

class Zer(Initializer):
    def __call__(self,shape,dtype=None):
        return np.zeros(shape)

class Mult(keras.layers.Layer):
    def __init__(self):
        super(Mult,self).__init__()
        return
    
    def build(self,input_shape):
        self.w = self.add_weight(shape=(input_shape[-1],),
                                      initializer=Zer(), name='w')
        self.built = True #??
        return
    
    def call(self,inputs):
        output = inputs*self.w
        return output

    # def compute_output_shape(self, input_shape): - do not need

inp = Input(shape=(5,))
mul = Mult()(inp)
#out = Lambda(lambda x: K.sqrt(K.sum(x*x)) )(mul)
out = Lambda(lambda x: K.sqrt(K.sum(x*x)+K.epsilon()) , output_shape=(1,) )(mul)

model = Model(inputs=inp,outputs=out)
model.compile(loss="mse",optimizer="sgd")
model.predict(np.array([[1,2,3,4,5]]))

array([0.00031623], dtype=float32)

In [51]:
getModelGradients(model,np.array([[1,1,1,1,1]]),np.array([[0]]))

[array([0., 0., 0., 0., 0.], dtype=float32)]

In [ ]:
class MinimalRNNCell(keras.layers.Layer):
    def __init__(self,units):
        self.units = units
        self.state_size = units #??
        super(MinimalRNNCell,self).__init__()
        return
    
    def build(self,input_shape):
        self.kernel = self.add_weight(shape=(input_shape[-1],self.units),
                                      initializer='uniform', name='kernel')
        self.r_kernel = self.add_weight(shape=(self.units,self.units),
                                      initializer='uniform', name='r_kernel')
        self.built = True #??
        return
    
    def call(self,inputs,states):
        prev_out = states[0]
        h = K.dot(inputs,self.kernel)
        output = h+K.dot(prev_out,self.r_kernel)
        return output, [output]


In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD,Adam

from pycallgraph import PyCallGraph
from pycallgraph.output import GraphvizOutput

model_data = np.random.rand(100,5)
model_labels = np.random.rand(100,10)


graphviz = GraphvizOutput()
graphviz.output_file = 'pycallgraph1.bmp'

with PyCallGraph(output=graphviz):
    model = Sequential((Dense(10,input_shape=(5,)),))
    optimizer = SGD(lr=0.01)

    model.compile(optimizer=optimizer,loss="mse")

    model.fit(model_data,model_labels,batch_size=10,verbose=1,epochs=1,validation_split=0.1)

In [ ]:
## Task 2
## Research Abilities to write custom layers, optimizers, losses
##

In [ ]:
## Task 3
## Research Abilities to make clusterized layers 
## (where elements in matrix are not independent)
##